In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
file  = open('data.txt', 'r', encoding = "utf8")

In [ ]:
lines=[]
for i in file:
    lines.append(i)

In [ ]:
data=""
for i in lines:
    data = ' '.join(lines)

In [ ]:
 data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [ ]:
data = data.split()
data = ' '.join(data)

In [ ]:
data[:1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [ ]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [ ]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [ ]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [ ]:
len(sequence_data)

105879

In [ ]:
vocab_size = len(tokenizer.word_index)+1

In [ ]:
vocab_size

8200

In [ ]:
sequence=[]
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

In [ ]:
len(sequence)

105876

In [ ]:
sequence = np.array(sequence)

In [ ]:
sequence

array([[   1, 1561,    5,  129],
       [1561,    5,  129,   34],
       [   5,  129,   34,  647],
       ...,
       [  28,    1, 8198, 8199],
       [   1, 8198, 8199, 3187],
       [8198, 8199, 3187, 3186]])

In [ ]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Building the LSTM Model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = 3))
model.add(LSTM(1000, return_sequences = True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation="softmax"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("next_words.keras", monitor="loss", verbose = 1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer = Adam(learning_rate = 0.001),metrics=['accuracy'])
model.fit(X, y, epochs=20, batch_size=32, callbacks=[checkpoint])

Epoch 1/20
3307/3309 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0568 - loss: 6.5425
Epoch 1: loss improved from inf to 6.28218, saving model to next_words.keras
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 93s 26ms/step - accuracy: 0.0568 - loss: 6.5423
Epoch 2/20
3308/3309 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0993 - loss: 5.7741
Epoch 2: loss improved from 6.28218 to 5.73831, saving model to next_words.keras
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 138s 25ms/step - accuracy: 0.0993 - loss: 5.7741
Epoch 3/20
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1181 - loss: 5.4737
Epoch 3: loss improved from 5.73831 to 5.44803, saving model to next_words.keras
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 142s 25ms/step - accuracy: 0.1181 - loss: 5.4737
Epoch 4/20
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1295 - loss: 5.2467
Epoch 4: loss improved from 5.44803 to 5.22693, saving model to next_words.keras
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 84s 25ms/step - accuracy: 0.1295 - loss: 5.2467
Epoch 

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

model = load_model('next_words.keras')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def predict_word(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds=np.argmax(model.predict(sequence))
    predicted_word = ""
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    print(predicted_word)
    return predicted_word

while True:
    text = input("Enter your line: ")
    if text=="1":
        break
    else:
        text = text.split(" ")
        text = text[-3:]
        predict_word(model, tokenizer, text)


Enter your line: hii how are
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
the
Enter your line: what
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
is
Enter your line: your
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
son
Enter your line: 1
